In [ ]:
from datetime import datetime

import numpy as np
import xarray as xr
import xesmf as xe
from siphon.catalog import TDSCatalog

In [ ]:
cf_roms = (
    ("x_wind_10m", "height4", "Uwind", "wind_time"),
    ("y_wind_10m", "height4", "Vwind", "wind_time"),
    ("integral_of_surface_net_downward_shortwave_flux_wrt_time", "height0", "swrad", "swrad_time"),  # accumulated
    ("specific_humidity_2m", "height1", "Qair", "qair_time"),
    ("air_temperature_2m", "height1", "Tair", "Tair_time"),  # Kelvin -> to Celsius
    ("precipitation_amount_acc", "height0", "rain", "rain_time"),  # accumulated
    ("air_pressure_at_sea_level", "height_above_msl", "Pair", "pair_time"),
    (
        "integral_of_surface_downwelling_longwave_flux_in_air_wrt_time",
        "height0",
        "lwrad_down",
        "lwrad_time",
    ),  # accumulated; units - 1 watt = 1 joule per second.
    ("cloud_area_fraction", "height3", "cloud", "cloud_time"),
)

In [ ]:
LAT_NEW = np.arange(55, 75, 0.02)
LON_NEW = np.arange(5, 45, 0.02)

def regrid(regridder, da):
    if regridder is None:
        target_grid = xr.Dataset(
            {
                "lat": (["lat"], LAT_NEW),
                "lon": (["lon"], LON_NEW)
            }
        )

        source_grid = xr.Dataset(
            {
                "lat": (("y", "x"), da.latitude.data),
                "lon": (("y", "x"), da.longitude.data),
            }
        )
        regridder = xe.Regridder(source_grid, target_grid, method="bilinear", unmapped_to_nan = True)
    return regridder, regridder(da)

In [ ]:
def generate_catalog_urls(start_year=2010, end_year=2020):
    hours = 0, 6, 12, 18
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            for day in range(1, 32):
                try:
                    datetime(year, month, day)  # validate date
                except ValueError:
                    continue
                for hour in hours:
                    yield (
                        datetime(year, month, day, hour), f"https://thredds.met.no/thredds/catalog/nora3/{year}/{month:02d}/{day:02d}/{hour:02d}/catalog.xml"
                    )

In [ ]:
dss = []
parameters = [x[0] for x in cf_roms]
for date_and_time, catalog_url in generate_catalog_urls():
    print(f"Processing {catalog_url}")
    cat = TDSCatalog(catalog_url)
    urls = [v.access_urls["opendap"] for k, v in cat.datasets.items() if "_fp" in k]
    ds = xr.open_mfdataset(urls, combine="by_coords", compat="no_conflicts", data_vars="all")[parameters]
    # filename = date_and_time.strftime("%Y%m%d_%H%M") + ".nc"
    # ds.to_netcdf(filename, mode="w")
    # comp = dict(zlib=True, complevel=5)
    # encoding = {var: comp for var in ds.data_vars}
    # ds.to_netcdf("foo.nc", encoding=encoding)
    dss.append(ds)
    if len(dss) > 1:
        break

In [ ]:
regridder = None

In [ ]:
ds = dss[1].copy(deep=True)

In [ ]:
da_x_wind_10m = ds["x_wind_10m"].isel(height4=0)
regridder, da_x_wind_10m = regrid(regridder, da_x_wind_10m)

In [ ]:
da_y_wind_10m = ds["y_wind_10m"].isel(height4=0)
regridder, da_y_wind_10m = regrid(regridder, da_y_wind_10m)

In [ ]:
da_swrad_acc = ds["integral_of_surface_net_downward_shortwave_flux_wrt_time"].isel(height0=0)
regridder, da_swrad_acc = regrid(regridder, da_swrad_acc)
da_swrad = da_swrad_acc.diff(dim="time") / (60 * 60)

In [ ]:
da_specific_humidity = ds["specific_humidity_2m"].isel(height1=0)
regridder, da_specific_humidity = regrid(regridder, da_specific_humidity)

In [ ]:
da_air_temperature = ds["air_temperature_2m"].isel(height1=0)
regridder, da_air_temperature = regrid(regridder, da_air_temperature)
da_air_temperature -= 273.15

In [ ]:
da_precipitation_acc = ds["precipitation_amount_acc"].isel(height0=0)
regridder, da_precipitation_acc = regrid(regridder, da_precipitation_acc)
da_precipitation = da_precipitation_acc.diff(dim="time") / (60 * 60)

In [ ]:
da_air_pressure = ds["air_pressure_at_sea_level"].isel(height_above_msl=0)
regridder, da_air_pressure = regrid(regridder, da_air_pressure)

In [ ]:
da_lwrad_acc = ds["integral_of_surface_downwelling_longwave_flux_in_air_wrt_time"].isel(height0=0)
regridder, da_lwrad_acc = regrid(regridder, da_lwrad_acc)
da_lwrad = da_lwrad_acc.diff(dim="time") / (60 * 60)

In [ ]:
da_cloud_area_fraction = ds["cloud_area_fraction"].isel(height3=0)
regridder, da_cloud_area_fraction = regrid(regridder, da_cloud_area_fraction)

In [ ]:
ds

In [ ]:
da_cloud_area_fraction.isel(time=3).plot()